# Classification: Cats VS Dogs

We downloaded the dataset from kaggle. The training archive contains 25,000 images of dogs and cats. 

# Using own data with included Datasets

In [1]:
import os
import mxnet as mx
import matplotlib.pyplot as plt
from mxnet import gluon
from mxnet import nd, autograd
import numpy as np
from mxnet.gluon.model_zoo import vision


In [2]:
def aug_transform(data, label):
    data = data.astype('float32')/255 # normalize the data
    size= 224
    aug = mx.image.ForceResizeAug((size,size)) # resize the image to 224x224
    data = aug(data)
    data=data.transpose((2, 0, 1))
    return data, label

# Reading Training date

In [3]:
data_folder = "data"
dataset_name = "CatsVsDogs"

training_path = os.path.join(data_folder, dataset_name)
print(training_path)

train_dataset = mx.gluon.data.vision.datasets.ImageFolderDataset(training_path,transform=aug_transform)



data/CatsVsDogs


In [4]:
sample_idx = 17990
sample = train_dataset[sample_idx]
data = sample[0]
label = sample[1]

#plt.imshow(data.asnumpy(), cmap='gray')
print("Data type: {}".format(data.shape))
print("Label: {}".format(label))
print("Label description: {}".format(train_dataset.synsets[label]))
assert label == 1

Data type: (3, 224, 224)
Label: 1
Label description: dog


# Test data

In [5]:
testing_path = os.path.join(data_folder, "{}_test".format(dataset_name))
print(testing_path)
test_dataset = mx.gluon.data.vision.datasets.ImageFolderDataset(testing_path,transform=aug_transform)

data/CatsVsDogs_test


In [6]:
sample_idx = 199

sample = test_dataset[sample_idx]
data = sample[0]
label = sample[1]

#plt.imshow(data.asnumpy(), cmap='gray')
print("Data type: {}".format(data.shape))
print("Label: {}".format(label))
print("Label description: {}".format(train_dataset.synsets[label]))

Data type: (3, 224, 224)
Label: 0
Label description: cat


# Batch size and DataLoader for train_data, test_dataset

In [7]:
batch_size = 32
train_data = gluon.data.DataLoader(train_dataset, batch_size, shuffle=True)
test_data = gluon.data.DataLoader(test_dataset,batch_size, shuffle=False)

# Initialize parameters

In [8]:
ctx = mx.gpu()
network=vision.resnet152_v2(pretrained=True, ctx=ctx)

# Fine-tuning pre-trained models

In [9]:
NUM_CLASSES = 2
with network.name_scope():
    network.output = gluon.nn.Dense(2)

In [10]:

network.collect_params().initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)

/home/ubuntu/anaconda3/envs/mxnet/lib/python3.6/site-packages/mxnet/gluon/parameter.py:320: UserWarning: Parameter resnetv20_batchnorm0_gamma is already initialized, ignoring. Set force_reinit=True to re-initialize.
  "Set force_reinit=True to re-initialize."%self.name)
/home/ubuntu/anaconda3/envs/mxnet/lib/python3.6/site-packages/mxnet/gluon/parameter.py:320: UserWarning: Parameter resnetv20_batchnorm0_beta is already initialized, ignoring. Set force_reinit=True to re-initialize.
  "Set force_reinit=True to re-initialize."%self.name)
/home/ubuntu/anaconda3/envs/mxnet/lib/python3.6/site-packages/mxnet/gluon/parameter.py:320: UserWarning: Parameter resnetv20_batchnorm0_running_mean is already initialized, ignoring. Set force_reinit=True to re-initialize.
  "Set force_reinit=True to re-initialize."%self.name)
/home/ubuntu/anaconda3/envs/mxnet/lib/python3.6/site-packages/mxnet/gluon/parameter.py:320: UserWarning: Parameter resnetv20_batchnorm0_running_var is already initialized, ignoring.

# Optimizer 

In [11]:
trainer = gluon.Trainer(network.collect_params(), 'adam', {'learning_rate': .0001})

# Softmax cross-entropy loss

In [12]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

# Evaluation Loop

In [13]:
def evaluate_accuracy(data_iterator, net):
    acc = mx.metric.Accuracy()
    curr_loss= []
    for d, l in data_iterator:
        data = d.as_in_context(ctx)
        label = l.as_in_context(ctx)
        output = net(data)
        
        loss = softmax_cross_entropy(output, label)
        
        predictions = nd.argmax(output, axis=1)
        acc.update(preds=[predictions], labels=[label])
        
        curr_loss.append(nd.mean(loss).asscalar())
        
        #print(acc)
    
    return acc.get()[1], np.average(curr_loss)

# Training loop

In [14]:
epochs = 10
smoothing_constant = .1
metric = mx.metric.Accuracy()

train_acc=[]
val_acc=[]

train_loss=[]
val_loss=[]


for e in range(epochs):
    metric.reset()
    for i, (d, l) in enumerate(train_data):
        data = d.as_in_context(ctx)
        label = l.as_in_context(ctx)
        with autograd.record():
            output = network(data)
            loss = softmax_cross_entropy(output, label)
        
        loss.backward()
        trainer.step(data.shape[0])
                
        ##########################
        #  Keep a moving average of the losses
        ##########################
        metric.update([label], [output])
        curr_loss = nd.mean(loss).asscalar()
        moving_loss = (curr_loss if ((i == 0) and (e == 0))
                       else (1 - smoothing_constant) * moving_loss + (smoothing_constant) * curr_loss)

        
        if i % 10 == 0 and i > 0:
                name, acc = metric.get()
                print('[Epoch %d Batch %d] Loss: %s Training %s=%f'%(e, i, moving_loss, name, acc))
           
    _, train_accuracy = metric.get()
    
    test_accuracy,test_loss = evaluate_accuracy(test_data, network)
    
    train_acc.append(train_accuracy)
    val_acc.append(test_accuracy)
    
    train_loss.append(moving_loss)
    val_loss.append(test_loss)
    
    #_, train_accuracy = metric.get()
    #print("Epoch %s. Loss: %s, Train_acc %s\n" % (e, moving_loss, train_accuracy))

    #test_accuracy = evaluate_accuracy(test_data, network)
    print("\nEpoch %s., Validation Accuracy %s,Validation Loss =%s \n" % (e, test_accuracy,val_loss[e]))

[Epoch 0 Batch 10] Loss: 0.3265344582383592 Training accuracy=0.928977
[Epoch 0 Batch 20] Loss: 0.1856434173116093 Training accuracy=0.941964
[Epoch 0 Batch 30] Loss: 0.12746406813525482 Training accuracy=0.947581
[Epoch 0 Batch 40] Loss: 0.12173914515400275 Training accuracy=0.950457
[Epoch 0 Batch 50] Loss: 0.11474262104692635 Training accuracy=0.954044
[Epoch 0 Batch 60] Loss: 0.11033477441916378 Training accuracy=0.955943
[Epoch 0 Batch 70] Loss: 0.07721241519461354 Training accuracy=0.959067
[Epoch 0 Batch 80] Loss: 0.0729287988174067 Training accuracy=0.961420
[Epoch 0 Batch 90] Loss: 0.07985138522478842 Training accuracy=0.961538
[Epoch 0 Batch 100] Loss: 0.07352247254280043 Training accuracy=0.962252
[Epoch 0 Batch 110] Loss: 0.06396595485255266 Training accuracy=0.963401
[Epoch 0 Batch 120] Loss: 0.059699738657245385 Training accuracy=0.964360
[Epoch 0 Batch 130] Loss: 0.054968010401567124 Training accuracy=0.965172
[Epoch 0 Batch 140] Loss: 0.06513032572966244 Training accura

KeyboardInterrupt: 

# Learning curve ( Plot Loss and Accuracy)

In [ ]:
plt.ion()
fig = plt.figure()
subfig=fig.add_subplot(122)
subfig.plot(train_acc, label = 'training')
if val_acc is not None:
    subfig.plot(val_acc, label = 'validation')
    
subfig.set_title('Model Accuracy')
subfig.set_xlabel('Epoch')
subfig.legend(loc='upper left')

subfig1=fig.add_subplot(121)
subfig1.plot(train_loss, label='training')
if val_loss is not None:
    subfig1.plot(val_loss, label = 'validation')
subfig1.set_title('Model Loss')
subfig1.set_xlabel('Epoch')
subfig1.legend(loc='upper left')
plt.ioff()


The plot shows after epoch 6 the training accuracy is high and the validation score is low, which shows it is overfitting the training data. The best accuracy we can get using Alnext is 90.8%.